In [7]:
import pandas as pd
import requests
import sqlalchemy
import sam_stuff
from datetime import timedelta, datetime
from pytz import timezone
from IPython.display import JSON

## Get connection to SQL

In [9]:
def get_connection_string():
    schema = "gans"
    host = "127.0.0.1"
    user = "root"
    password = sam_stuff.my_sql_password
    port = 3306
    
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return connection_string

## Get Weather

In [11]:
berlin_timezone = timezone('Europe/Berlin')
API_key = sam_stuff.my_API_key

In [13]:
def get_city():
    con_string = get_connection_string()
    city_df = pd.read_sql('city',con=con_string)
    return city_df

In [15]:
city_df = get_city()

In [17]:
city_df

,city_id,city,country,population,latitude,longitude
0,1,Berlin,Germany,3755251,52°31′12″N,13°24′18″E
1,2,London,England,8799800,51°30′26″N,0°7′39″W


In [43]:
#create the function:

def get_and_store_weather_data(city_df):
# create a dictionary with keys (same as sql) with empty list inside before the loop.
    weather_items = {
        "city_id":[],  #*
        "city": [],
        "forecast_time": [],
        "temperature": [],
        "feels_like": [],
        "forecast": [],
        "rain_in_last_3h": [],
        "wind_speed": [],
        "data_retrieved_at": [],
    }
# do url request for so many cities as we have in the list.
    for i, row in city_df.iterrows():
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={row['city']}&appid={API_key}&units=metric")
        # instead of give "city", we need to go inside the row and get the city name. {row["city"]}
        response = requests.get(url)
        weather_json = response.json()
        my_timestamp = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")   # we will not change if it is london, in this case, we dont care

        # for loop throu 
        for item in weather_json["list"][:8]:
            weather_items['city_id'].append(row['city_id']),  #*
            weather_items['city'].append(row['city']),  #*
            weather_items["forecast_time"].append(item.get('dt_txt',None)),
            weather_items["temperature"].append(item["main"].get("temp", None)),
            weather_items["feels_like"].append(item["main"].get("feels_like", None)),
            weather_items["forecast"].append(item["weather"][0].get("main", None)),
            weather_items["rain_in_last_3h"].append(item.get("rain", {}).get("3h", 0)),
            weather_items["wind_speed"].append(item["wind"].get("speed", None)),
            weather_items['data_retrieved_at'].append(my_timestamp)
  
    weather_df = pd.DataFrame(weather_items)

    #* when we create this weather_df, we cant connect it to our primary key in our city_df table in sql. As in sql city table, the primary key is city_id.
    #* So we need this information to be able to connect weather_df to city table.  
    #* so we need to add "city_id" to our dictionary.
    #* We append also the "city_id" in our for loop. And we get it from city_df: instead of city_id, we get (row["city_id"])
 
    # to send it to sql:
    my_con = get_connection_string()
    weather_df.to_sql('weather',if_exists='append',con=my_con,index=False)

In [39]:
get_and_store_weather_data(city_df)

## Get Flights

In [12]:
def get_airports():
    connection_string = get_connection_string()
    airports_df = pd.read_sql('airport',con=connection_string)
    return airports_df

In [13]:
airports_df = get_airports()
airports_df

,ICAO,city_id
0,EDDB,1
1,EGLL,2


In [14]:
def get_and_store_flights(airports_df):

    #connection_string = get_connection_string()
    
    now = datetime.now()
    tomorrow = now + timedelta(days=1)
    tomorrow = tomorrow.strftime('%Y-%m-%d')
    
    times_lists = [{'from':'00:00','to':'11:59'},{'from':'12:00','to':'23:59'}]
    
    flights_dict = {
            'ICAO':[],
            'from_city':[],
            'arrival_time':[],
            'airline':[],
            'terminal':[],
            'flight_status':[]}
    
    for i, row in airports_df.iterrows():
        for time in times_lists:
            
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{row['ICAO']}/{tomorrow}T{time['from']}/{tomorrow}T{time['to']}"
            querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
            headers = {
                "X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            response = requests.get(url, headers=headers, params=querystring)
            
            my_json = response.json()
            
            for flight in my_json['arrivals']:
                flights_dict['ICAO'].append(row['ICAO'])
                flights_dict['arrival_time'].append(flight['arrival'].get('scheduledTime',{}).get('local',None))
                flights_dict['from_city'].append(flight['departure'].get('airport',{}).get('name','unknown'))
                flights_dict['airline'].append(flight['airline'].get('name',None))
                flights_dict['terminal'].append(flight['arrival'].get('terminal','unknown'))
                flights_dict['flight_status'].append(flight.get('status',None))
        
    flight_df = pd.DataFrame(flights_dict)
    flight_df['arrival_time'] = flight_df['arrival_time'].str[:-6]
    flight_df.to_sql('flight',con=connection_string,index=False,if_exists='append')

In [15]:
get_and_store_flights(get_airports())

NameError: name 'connection_string' is not defined